#### Jalaj Bansal, Manisha Kataria, Aditya Agarwal
### A1 - C
### Perceptron Algorithm

In [1]:
# libs
import pandas as pd
import numpy as np
import math

## Dataset LP 1

In [2]:
# Read data and normalize if required

df = pd.read_csv('dataset_LP_1.txt',sep=',',header=None)
df.columns = ['x1','x2','x3','x4','t']
df.insert(0,'x0',1)
print(df.head())


def normalize(ds):
    for itr in range(ds.shape[1]):
        if ds.columns[itr] == 't' or ds.columns[itr] == 'x0' : continue
        original = ds.iloc[:,itr]
        ds.iloc[:,itr] = (original - np.mean(original))/np.std(original)
    return ds

   x0       x1      x2      x3       x4  t
0   1  3.62160  8.6661 -2.8073 -0.44699  0
1   1  4.54590  8.1674 -2.4586 -1.46210  0
2   1  3.86600 -2.6383  1.9242  0.10645  0
3   1  3.45660  9.5228 -4.0112 -3.59440  0
4   1  0.32924 -4.4552  4.5718 -0.98880  0


#### Our perceptron model is w0 + w1x1 + w2x2 + w3x3 + w4x4, if the equation throws activation>0 then t = 1 else t = 0
#### 1. We normalise our dataset (OPTIONAL)
#### 2. We split the data into 70:30 train:test
#### 3. We build our hypthesis by learning the weights (parameters) by applying Stochastic Gradient descent
#### 4. Using the parameters we make predictions and report accuracy on test data as % correctly classified / total samples

## Dataset LP 2

In [4]:
df = pd.read_csv('dataset_LP_2.csv',sep=',',header=None)
df.columns = ['x1','x2','x3','t']
df.insert(0,'x0',1)
print(df.head())

def normalize(ds):
    for itr in range(ds.shape[1]):
        if ds.columns[itr] == 't' or ds.columns[itr] == 'x0' : continue
        original = ds.iloc[:,itr]
        ds.iloc[:,itr] = (original - np.mean(original))/np.std(original)
    return ds

   x0        x1        x2        x3  t
0   1 -6.672418 -1.206198 -1.081050  0
1   1  1.675598  0.614994 -0.971600  0
2   1 -4.039058  0.335102  0.544618  1
3   1  0.793526 -0.235277  0.551771  1
4   1  3.820273 -0.274691  0.454743  1


#### Our perceptron model is w0 + w1x1 + w2x2 + w3x3, if the equation throws activation>0 then t = 1 else t = 0
#### 1. We normalise our dataset (OPTIONAL)
#### 2. We split the data into 70:30 train:test
#### 3. We build our hypthesis by learning the weights (parameters) by applying Stochastic Gradient descent
#### 4. Using the parameters we make predictions and report accuracy on test data as % correctly classified / total samples

### Learning the weights with Stochastic Gradient Descent

In [5]:
# Misclassified points / Total points
def cost_find(w,y,x):
    n = len(y)
    y = y[:,np.newaxis]
    y_pred = x @ w
    misclass = 0
    for i in range(0,len(y_pred)):
        if y_pred[i]>0:
            y_pred[i]=1
        else:
            y_pred[i]=0
        if y_pred[i]!=y[i]:
            misclass += 1
    #print("Total Misclassified : ",misclass)
    cost = (misclass/n) * 100.0
    #print("% Accuracy : ", 100-cost)
    return cost

# Since this is Stochastic, it picks a random point from the training data and checks if it is correctly classified
# w(t+1)= w(t) + learning_rate * (expected(t) - predicted(t)) * x(t)
def SCostFunction(w, x, y):
    n = len(y)
    y = y[:,np.newaxis]
    # random sampling
    i = np.random.choice(np.arange(len(y)))
    x_temp = x[i]
    x_temp = x_temp.reshape(1,x_temp.shape[0]) # xT
    y_pred = x_temp @ w # xTw
    if y_pred > 0:
        y_pred = 1
    else :
        y_pred = 0
    x_temp = x_temp.reshape(x_temp.shape[1],1)
    y1 = y_pred - y[i]
    gradient = (x_temp @ y1) # expected t - predicted t * x
    gradient = gradient.reshape(gradient.shape[0],1)
    return gradient


# perform 1000000 iterations where weight is updated on misclassification
# stop early if zero misclassified points
def Stochastic_GD(w,x,y,epochs,eta,threshold):
    n=len(y)
    itrs=0
    costs = []
    while True:
        gradient = SCostFunction(w,x,y)
        cost = cost_find(w,y,x)
        w = w - (eta * gradient) # update weights - for correct class there is no change as gradient is zero
        itrs=itrs+1
        if itrs%100 == 0:
            print("No of iterations: ",itrs)
            print("Training error: ",cost)    
        if itrs==epochs  or cost==0.0:
            print("No of iterations: ",itrs)    
            print("Training error: ",cost)
            return cost,w
        costs.append(cost)

# helper        
def find_weights(x_train,y_train,initial_w,eta,epochs,threshold):
    tcost,final_w = Stochastic_GD(initial_w,x_train,y_train,epochs,eta,threshold)
    return final_w


# 70:30 train:test split
def split_data(df):
    train,test = np.split(df.sample(frac=1).reset_index(drop=True),[int(.7*len(df))])
    return train,test

# start perceptron with input parameters
def perceptron(dataset,learning_rate,norm,epochs,threshold):
    if norm==1:
        dataset = normalize(dataset)
    train,test = split_data(dataset)
    print(train.shape[1])
    x_train = np.array(train.iloc[:,0:train.shape[1]-1])
    y_train = np.array(train.iloc[:,train.shape[1]-1])
    x_test = np.array(test.iloc[:,0:train.shape[1]-1])
    y_test = np.array(test.iloc[:,train.shape[1]-1])
    # initialize random weights
    initial_w = np.random.rand(train.shape[1]-1,1)
    # print(initial_w)
    final_w = find_weights(x_train,y_train,initial_w,learning_rate,epochs,threshold)
    # print(final_w)
    test_error = cost_find(final_w,y_test,x_test)
    print("Testing accuracy :",100.0-test_error,"%")
    return final_w,x_test,y_test

# get predicted values for inputs
def predict(parameters,x):
    y_pred = x @ parameters
    for i in range(0,len(y_pred)):
        if y_pred[i]>0:
            y_pred[i]=1
        else:
            y_pred[i]=0
    return y_pred

## begin
model_parameters,x_test,y_test = perceptron(df,0.1,1,1000,0)

#print(predict(model_parameters,x_test))

5
No of iterations:  55
Training error:  0.0
Testing accuracy : 100.0 %
